In [ ]:
import os, torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig, TrainingArguments, pipeline
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from datasets import Dataset
from dotenv import load_dotenv
from IPython.display import Markdown, display
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.vectorstores import Qdrant
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

In [ ]:
# You need to have a .env file in the same directory
# It should have your Hugging Face Token key in the below format
# HF_TOKEN = "your_key_for_Hugging_Face"

load_dotenv()
model = "google/gemma-2b-it"

bnbConfig = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model, quantization_config=bnbConfig, device_map="auto", token=os.environ['HF_TOKEN'])

model = AutoModelForCausalLM.from_pretrained(
    model,
    device_map = "auto",
    quantization_config=bnbConfig,
    token=os.environ['HF_TOKEN']
)

In [ ]:
system =  "You are a skilled software engineer who consistently produces high-quality Python code."
user = "Write a Python code to display text in a star pattern."

prompt = f"System: {system} \n User: {user} \n AI: "
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, num_return_sequences=1, max_new_tokens=1000)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)
Markdown(text.split("AI:")[1])

In [ ]:
data = pd.read_csv("<path_to_dataset>")
dataset = Dataset.from_pandas(data)
data.head()

In [ ]:
def formatting_func(example):
    template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
    line = template.format(instruction=example['Question'], response=example['Answer'])
    return [line]

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    max_seq_length=512,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=50,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

In [ ]:
trainer.train()

In [ ]:
system =  "You are a skilled software engineer who consistently produces high-quality Python code."
question = system + "What is the difference between a variable and an object"

prompt = f"Question: {question} \n Answer: "
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, num_return_sequences=1, max_new_tokens=512)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

Markdown(text.split("Answer:")[1])

In [ ]:
pdf_loader = PyPDFDirectoryLoader("<path_to_PDF_Documents_Directory>")
pdfs = pdf_loader.load()

In [ ]:
# HuggingFaceEmbeddings class
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device": "cuda"}
)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
all_splits = text_splitter.split_documents(pdfs)

In [ ]:
# Storing Vectom Embeddings in qdrant DB
# Can also use ChromaDB, PyMongo, AstraDB
qdrant_collection = Qdrant.from_documents(
    all_splits,               
    embeddings,                # HuggingFaceEmbeddings object
    location=":memory:",      
    collection_name="all_documents" 
)

In [ ]:
# Retriever Object
retriever = qdrant_collection.as_retriever()

In [ ]:
# Token length max possible is around 8096 token
pipeline = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    model_kwargs = {"torch.dtype": torch.bfloat16},
    max_new_tokens=512    
)

In [ ]:
question = "Tell me about SMCC"

message = [
    {"role": "user", "content": question},
]

prompt = pipeline.tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)

outputs = pipeline(
    prompt,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_k=10,
    top_p=0.95
)
Markdown(outputs[0]["generated_text"][len(prompt):])

In [ ]:
gemma_llm = HuggingFacePipeline(
    pipeline=pipeline,
    model_kwargs={
        "temperature": 0.7,
        "max_new_tokens": 512,
        "add_special_tokens": True,
        "do_sample": True,
        "top_k": 10,
        "top_p": 0.95
    },
)
# Create a RetrievalQA object
qa = RetrievalQA.from_chain_type(
    llm=gemma_llm,  # Pass the text-generation pipeline object
    chain_type="stuff",
    retriever=retriever  # retriever object
)

In [ ]:
question = "Tell me about SMCC"
message = [
    {"role": "user", "content": question},
]

prompt = pipeline.tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True, truncation=True)
result = qa.invoke(prompt)
Markdown(result['result'].split('Helpful Answer:')[1])